#Install lib

In [4]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
#!apt install git-lfs

#Enviroment

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible answers are allowed or not).
squad_v2 = True
#model_checkpoint = "xlm-roberta-base"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

#Preprocessing the training data

In [9]:
import json
from pathlib import Path
import torch

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [24]:
def read_squad_train(path,data_index,context_size):
  """
  Read data format squad 
  -> return contexts, questions, answers of sentenses
  """
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  size=0
  for group in squad_dict['data'][data_index-1:data_index]:
      for passage in group['paragraphs']:
        if size <= context_size:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                if answer['answer_start']>=0:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)
          size+=1
        else: break

  return contexts, questions, answers
def read_squad_val(path):
  """
  Read data format squad 
  -> return contexts, questions, answers of sentenses
  """
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  for group in squad_dict['data']:
      for passage in group['paragraphs']:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)

  return contexts, questions, answers
def add_token_positions(encodings, answers):
  """
  Convert answer start- end from string to token
  """
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
    # if None, the answer passage has been truncated
    if start_positions[-1] is None:
        start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
        end_positions[-1] = tokenizer.model_max_length
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
def add_end_idx(answers, contexts):
  """
  Squad format chỉ có id start của câu trả lời
  -> Hàm này dùng để tạo id end của câu trả lời
  """
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two – fix this
    if context[start_idx:end_idx] == gold_text:
        answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters


def read_squad_train2(path,context_index):
  """
  Read data format squad 
  -> return contexts, questions, answers of sentenses
  """
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  for group in squad_dict['data'][0:1]:
      for passage in group['paragraphs'][context_index[0]:context_index[1]+1]:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                if answer['answer_start']>=0:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)
  return contexts, questions, answers
train_contexts, train_questions, train_answers = read_squad_train2('/content/drive/MyDrive/DoAnCK/Dataset/squad_vi.json', [10000,15000])
val_contexts, val_questions, val_answers = read_squad_train2('/content/drive/MyDrive/DoAnCK/Dataset/squad_vi.json', [17000,18000])
#val_contexts, val_questions, val_answers = read_squad_train('/content/drive/MyDrive/CS221/DoAnCK/Dataset/squad_vi.json', 3000, 3500)
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts) 

#train
new_train_contexts=[]
new_train_questions=[]
new_train_answers=[]
for i in range(len(train_answers)):
  if train_answers[i]['answer_start']!=train_answers[i]['answer_end']:
    new_train_contexts.append(train_contexts[i])
    new_train_questions.append(train_questions[i])
    new_train_answers.append(train_answers[i])
(train_contexts, train_questions, train_answers)=(new_train_contexts, new_train_questions, new_train_answers)
(new_train_contexts, new_train_questions, new_train_answers)=(0,0,0)
new_val_contexts=[]
new_val_questions=[]
new_val_answers=[]
for i in range(len(val_answers)):
  if val_answers[i]['answer_start']!=val_answers[i]['answer_end']:
    new_val_contexts.append(val_contexts[i])
    new_val_questions.append(val_questions[i])
    new_val_answers.append(val_answers[i])
(val_contexts, val_questions, val_answers)=(new_val_contexts, new_val_questions, new_val_answers)
(new_val_contexts, new_val_questions, new_val_answers)=(0,0,0)
# #chia train val
# val_contexts, val_questions, val_answers = train_contexts[12000:], train_questions[12000:], train_answers[12000:]
# train_contexts, train_questions, train_answers = train_contexts[:12000], train_questions[:12000], train_answers[:12000]


train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [25]:
len(train_answers)

19861

Tải data lên pytorch

In [26]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [27]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

#Load pretrain

In [15]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model.load_state_dict(torch.load('/content/drive/MyDrive/LOG/sublog3/checkpoint-10292/pytorch_model.bin'))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

<All keys matched successfully>

#Train

In [16]:
from torch.utils.data import DataLoader
from transformers import AdamW

In [18]:
val_count = len(val_dataset)

In [19]:
epochs = 10
batch_size=16
train_count = len(train_dataset)
val_count = len(val_dataset)
PATH = "//content/drive/MyDrive/LOG/best_model.model"

###Train with trainer

###10 epoch

In [20]:
import numpy as np

In [ ]:
args = TrainingArguments(
    f"/content/drive/MyDrive/LOG/sublog2",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [21]:
def f1_score(start_pred,end_pred,start_true,end_true):
  a=np.arange(int(start_pred),int(end_pred+1), step=1)
  b=np.arange(int(start_true),int(end_true+1), step=1)
  true_count = len(np.intersect1d(a,b))
  if ((int(end_true)-int(start_true)) == 0):
    recall = 0
  else:
    recall = true_count/(int(end_true)-int(start_true))
  if ((int(end_pred)-int(start_pred)) == 0):
    precision = 0
  else:
    precision= true_count/(int(end_pred)-int(start_pred))
  if (precision + recall == 0):
    return 0
  f1=2*(precision*recall)/(precision+recall)
  return f1

In [22]:
def compute_metrics(eval_pred):    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    count = 0
    for i in range(len(predictions[0])):
      if (predictions[0][i] == labels[0][i] and predictions[1][i] == labels[1][i]):
        count += 1
    f1 = 0
    for i in range(len(predictions[0])):
      f1 += f1_score(predictions[0][i], predictions[1][i], labels[0][i], labels[1][i])

    return {'exact_match': round(count*100/len(predictions[0]), 3), 'f1_score': round(f1/len(predictions[0]),3)} #metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#0-10000


In [ ]:
args = TrainingArguments(
    f"/content/drive/MyDrive/LOG/sublog3",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy = "epoch",
    #push_to_hub=True,
)
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 41166
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 25730
  Number of trainable parameters = 66364418


Epoch,Training Loss,Validation Loss,Exact Match,F1 Score
1,2.415900,2.505397,28.284000,0.463000
2,1.864500,2.454407,30.599000,0.503000


***** Running Evaluation *****
  Num examples = 3974
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/sublog3/checkpoint-2573
Configuration saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-2573/config.json
Model weights saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-2573/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-2573/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-2573/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3974
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/sublog3/checkpoint-5146
Configuration saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-5146/config.json
Model weights saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-5146/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-5146/tokenizer_config.json
Special tokens file saved in /cont

Epoch,Training Loss,Validation Loss,Exact Match,F1 Score
1,2.415900,2.505397,28.284000,0.463000
2,1.864500,2.454407,30.599000,0.503000
3,1.514800,2.444758,31.152000,0.522000
4,1.189400,2.676967,31.480000,0.528000
5,0.885300,2.908550,30.649000,0.521000


***** Running Evaluation *****
  Num examples = 3974
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/sublog3/checkpoint-7719
Configuration saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-7719/config.json
Model weights saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-7719/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-7719/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-7719/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3974
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/sublog3/checkpoint-10292
Configuration saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-10292/config.json
Model weights saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-10292/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/LOG/sublog3/checkpoint-10292/tokenizer_config.json
Special tokens file saved in /

In [23]:
args = TrainingArguments(
    f"/content/drive/MyDrive/LOG//content/drive/MyDrive/LOG/sublog5",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy = "epoch",
    #push_to_hub=True,
)
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 41166
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12865
  Number of trainable parameters = 66364418
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Exact Match,F1 Score
1,0.814100,3.321112,29.567000,0.493000
2,0.516000,3.559578,29.391000,0.506000


***** Running Evaluation *****
  Num examples = 3974
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/sublog4/checkpoint-2573
Configuration saved in /content/drive/MyDrive/LOG/sublog4/checkpoint-2573/config.json
Model weights saved in /content/drive/MyDrive/LOG/sublog4/checkpoint-2573/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/LOG/sublog4/checkpoint-2573/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/LOG/sublog4/checkpoint-2573/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3974
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/sublog4/checkpoint-5146
Configuration saved in /content/drive/MyDrive/LOG/sublog4/checkpoint-5146/config.json
Model weights saved in /content/drive/MyDrive/LOG/sublog4/checkpoint-5146/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/LOG/sublog4/checkpoint-5146/tokenizer_config.json
Special tokens file saved in /cont

KeyboardInterrupt: ignored

#10000-15000

In [28]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model.load_state_dict(torch.load('/content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5/checkpoint-3726/pytorch_model.bin'))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

<All keys matched successfully>

In [29]:
args = TrainingArguments(
    f"/content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy = "epoch",
    #push_to_hub=True,
)
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 19861
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12420
  Number of trainable parameters = 66364418


Epoch,Training Loss,Validation Loss,Exact Match,F1 Score
1,2.389000,2.190277,32.612000,0.539000
2,1.833300,2.231283,32.788000,0.543000
3,1.422200,2.413545,32.964000,0.553000
4,1.006200,2.693565,32.813000,0.547000
5,0.753200,3.050770,32.134000,0.538000
6,0.521200,3.398000,31.731000,0.535000
7,0.381800,3.784125,31.681000,0.535000
8,0.246800,4.068014,31.782000,0.540000
9,0.192300,4.363337,32.260000,0.541000
10,0.129900,4.653668,31.706000,0.538000


***** Running Evaluation *****
  Num examples = 3974
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5/checkpoint-1242
Configuration saved in /content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5/checkpoint-1242/config.json
Model weights saved in /content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5/checkpoint-1242/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5/checkpoint-1242/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5/checkpoint-1242/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3974
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5/checkpoint-2484
Configuration saved in /content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5/checkpoint-2484/config.json
Model weights saved in /content/drive/MyDrive/LO

TrainOutput(global_step=12420, training_loss=0.8826650794577483, metrics={'train_runtime': 10509.9002, 'train_samples_per_second': 18.897, 'train_steps_per_second': 1.182, 'total_flos': 2.594901219735552e+16, 'train_loss': 0.8826650794577483, 'epoch': 10.0})

In [ ]:
model2 = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model2.load_state_dict(torch.load('/content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5/checkpoint-3726/pytorch_model.bin'))

In [32]:
args = TrainingArguments(
    f"/content/drive/MyDrive/LOG/content/drive/MyDrive/LOG/sublog5",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy = "epoch",
    #push_to_hub=True,
)
trainer = Trainer(
    model2,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [33]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3974
  Batch size = 16


{'eval_loss': 2.4135451316833496,
 'eval_exact_match': 32.964,
 'eval_f1_score': 0.553,
 'eval_runtime': 67.319,
 'eval_samples_per_second': 59.032,
 'eval_steps_per_second': 3.699}

Evaluation